In [48]:
import torch
import tiktoken

batch_size = 3
tokenizer = tiktoken.encoding_for_model('gpt2')
prefixes = ['a', 'b', 'cc', 'ddddd']

prefixes = tokenizer.encode_batch(prefixes)
max_len = max([len(i) for i in prefixes])
padded_prefixes = [[tokenizer.eot_token] * (max_len - len(i)) + i for i in prefixes]

batch_rem = len(prefixes) % batch_size
batch_padding = torch.ones(size=(batch_size - batch_rem, max_len))
batch_padding = batch_padding.masked_fill(batch_padding == 1, tokenizer.eot_token)

padded_prefixes = torch.tensor(padded_prefixes)
batch_prefixes = torch.cat((padded_prefixes, batch_padding), dim = 0)
batch_prefixes = batch_prefixes.reshape(-1, batch_size, max_len)

attn_mask = torch.ones_like(batch_prefixes)
attn_mask = attn_mask.masked_fill(batch_prefixes == tokenizer.eot_token, 0)


In [49]:
batch_prefixes

tensor([[[50256., 50256.,    64.],
         [50256., 50256.,    65.],
         [50256., 50256.,   535.]],

        [[ 1860.,  1860.,    67.],
         [50256., 50256., 50256.],
         [50256., 50256., 50256.]]])

In [50]:
attn_mask

tensor([[[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [31]:
padded_prefixes = torch.tensor(padded_prefixes)

In [33]:
padded_prefixes.shape

torch.Size([4, 3])

tensor([[[50256., 50256.,    64.],
         [50256., 50256.,    65.],
         [50256., 50256.,   535.]],

        [[ 1860.,  1860.,    67.],
         [50256., 50256., 50256.],
         [50256., 50256., 50256.]]])

In [44]:
attn_mask = torch.ones_like(batch_prefixes)
attn_mask = attn_mask.masked_fill(batch_prefixes == tokenizer.eot_token, 0)

In [45]:
attn_mask

tensor([[[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [52]:
import torch
import torch.nn.functional as F

In [59]:
prefixes = [[0, 0, 1], 
                [0, 0, 2], 
                [1, 2, 3], 
                [0, 3, 4]]
batch_size = 3
pad_token = 0

In [60]:
max_len = max([len(i) for i in prefixes])
prefixes = torch.tensor([[pad_token] * (max_len - len(i)) + i for i in prefixes])

pad_size = batch_size - (len(prefixes) % batch_size) if (len(prefixes) % batch_size) != 0 else 0
batch_pad = torch.empty(size = (pad_size, max_len)).fill_(pad_token)
prefixes = torch.cat((prefixes, batch_pad))

In [61]:
prefixes

tensor([[0., 0., 1.],
        [0., 0., 2.],
        [1., 2., 3.],
        [0., 3., 4.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [62]:
prefixes = torch.tensor(prefixes).reshape(-1, batch_size, max_len)
prefixes

<ipython-input-62-1d3620390b49>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prefixes = torch.tensor(prefixes).reshape(-1, batch_size, max_len)


tensor([[[0., 0., 1.],
         [0., 0., 2.],
         [1., 2., 3.]],

        [[0., 3., 4.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [68]:
attention_mask = torch.ones_like(prefixes).masked_fill(prefixes == pad_token, 0)
attention_mask

tensor([[[0., 0., 1.],
         [0., 0., 1.],
         [1., 1., 1.]],

        [[0., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [69]:
for idx, batch in enumerate(prefixes):
    for i in range(2):
        probs = F.softmax(batch, dim = -1)
        probs = probs * attention_mask[idx]
        print(f"probs: {probs}")
        next_token = torch.multinomial(probs, num_samples=1)
        batch = torch.cat((batch, next_token), dim = 1)
    print(batch.shape)
    

probs: tensor([[0.0000, 0.0000, 0.5761],
        [0.0000, 0.0000, 0.7870],
        [0.0900, 0.2447, 0.6652]])


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 1

In [65]:
next_token

tensor([[1],
        [3],
        [3]])

In [66]:
print(batch)

tensor([[0., 3., 4., 2., 1.],
        [0., 0., 0., 1., 3.],
        [0., 0., 0., 0., 3.]])
